In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



#postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
#postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
#avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')




In [3]:
#cargamos zips
avisosDetalleZIP = pd.read_csv('resources/zip/fiuba_6_avisos_detalle.csv')
avisosDetalleMissing = pd.read_csv('resources/zip/fiuba_6_avisos_missing.csv')

In [4]:
avisosDetallesCompletos = pd.concat([avisosDetalleZIP,avisosDetalleMissing])

In [5]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

In [6]:
postulacionesMerged = pd.merge(postulaciones,avisosDetalle,on='idaviso',how='inner');

In [7]:
postulacionesMerged['datePostulacion'] = pd.DatetimeIndex(postulacionesMerged['fechapostulacion']).date
postulacionesMergedFiltered = postulacionesMerged.drop(['ciudad','mapacalle','idpais','fechapostulacion'],axis=1)

In [5]:
postulacionesMergedFiltered.columns

Index(['idaviso', 'idpostulante', 'titulo', 'descripcion', 'nombre_zona',
       'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa', 'datePostulacion'],
      dtype='object')

In [8]:
postulacionesMerged['cantidadEmpresa'] = 1;
cantAvisosPorEmpresa = postulacionesMerged.groupby('denominacion_empresa').sum()
cantAvisosPorEmpresa = cantAvisosPorEmpresa.reset_index()
cantAvisosPorEmpresa = cantAvisosPorEmpresa[['denominacion_empresa','cantidadEmpresa']]

In [9]:
postulacionesConEmpresas = pd.merge(postulacionesMergedFiltered,cantAvisosPorEmpresa,how='inner',on='denominacion_empresa')

In [10]:
postulacionesConEmpresas.columns

Index(['idaviso', 'idpostulante', 'titulo', 'descripcion', 'nombre_zona',
       'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa', 'datePostulacion', 'cantidadEmpresa'],
      dtype='object')

In [11]:
postulacionesFinal = postulacionesConEmpresas.drop('titulo',axis=1)
postulacionesFinal.drop(['descripcion','denominacion_empresa'],axis=1,inplace=True);
postulacionesFinal.drop('nombre_area',axis=1,inplace=True)

In [68]:
postulacionesFinal.head()

,idaviso,idpostulante,nombre_zona,tipo_de_trabajo,nivel_laboral,datePostulacion,cantidadEmpresa
0,1112248724,NjlD,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-19,2318
1,1112248724,MV6eM8N,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-30,2318
2,1112248724,5LzQkN,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-15,2318
3,1112248724,8M2rOBz,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-30,2318
4,1112248724,Dr6V3Da,Gran Buenos Aires,Full-time,Senior / Semi-Senior,2018-01-23,2318


In [12]:
postulacionesOHC = pd.concat([postulacionesFinal,pd.get_dummies(postulacionesFinal['nombre_zona'], prefix='zona')],axis=1)
postulacionesOHC = pd.concat([postulacionesOHC,pd.get_dummies(postulacionesFinal['tipo_de_trabajo'], prefix='tipo')],axis=1)
postulacionesOHC = pd.concat([postulacionesOHC,pd.get_dummies(postulacionesFinal['nivel_laboral'], prefix='nivel')],axis=1)

In [13]:
postulacionesOHC.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona'],axis=1,inplace=True)

In [14]:
postulacionesOHC['sepostulo'] = 1;

In [55]:
postulacionesOHC.info

<bound method DataFrame.info of             idaviso idpostulante  cantidadEmpresa_x  cantidadEmpresa_y  \
0        1112248724         NjlD                  1               2318   
1        1112248724      MV6eM8N                  1               2318   
2        1112248724       5LzQkN                  1               2318   
3        1112248724      8M2rOBz                  1               2318   
4        1112248724      Dr6V3Da                  1               2318   
5        1112248724       8mkDQD                  1               2318   
6        1112248724      VNk6V22                  1               2318   
7        1112248724      dYjLleA                  1               2318   
8        1112248724      xkadArb                  1               2318   
9        1112248724      xkvx9D0                  1               2318   
10       1112248724        aj1v0                  1               2318   
11       1112248724        6OkGl                  1               2318   
12    

In [15]:
postulacionesOHC.columns

Index(['idaviso', 'idpostulante', 'datePostulacion', 'cantidadEmpresa',
       'zona_Capital Federal', 'zona_GBA Oeste', 'zona_Gran Buenos Aires',
       'tipo_Fines de Semana', 'tipo_Full-time', 'tipo_Part-time',
       'tipo_Pasantia', 'tipo_Por Contrato', 'tipo_Por Horas',
       'tipo_Primer empleo', 'tipo_Teletrabajo', 'tipo_Temporario',
       'tipo_Voluntario', 'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'sepostulo'],
      dtype='object')

In [16]:
avisosPostulados = postulacionesOHC[['idaviso','idpostulante','datePostulacion']]

In [73]:
avisosPostulados.info

<bound method DataFrame.info of             idaviso idpostulante datePostulacion
0        1112248724         NjlD      2018-01-19
1        1112248724      MV6eM8N      2018-01-30
2        1112248724       5LzQkN      2018-01-15
3        1112248724      8M2rOBz      2018-01-30
4        1112248724      Dr6V3Da      2018-01-23
5        1112248724       8mkDQD      2018-01-21
6        1112248724      VNk6V22      2018-01-15
7        1112248724      dYjLleA      2018-01-17
8        1112248724      xkadArb      2018-01-15
9        1112248724      xkvx9D0      2018-01-22
10       1112248724        aj1v0      2018-01-17
11       1112248724        6OkGl      2018-01-22
12       1112248724       686xdx      2018-01-24
13       1112248724      lDw9dxz      2018-01-15
14       1112248724      NzdJNoJ      2018-01-17
15       1112248724      8MaOpPD      2018-01-16
16       1112248724       aE15QW      2018-01-15
17       1112248724       E9jRRJ      2018-01-18
18       1112248724      mzVJE43     

In [17]:
avisosPostulados['cantPostulaciones'] = 1;
avisosPostuladosGP = avisosPostulados.groupby(['idaviso','idpostulante']).sum()
avisosPostuladosGP = avisosPostuladosGP.reset_index()
avisosPostuladosGP.info

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


<bound method DataFrame.info of             idaviso idpostulante  cantPostulaciones
0           8725750       1qwDrO                  1
1           8725750      2zkzE9Y                  1
2           8725750      2zwqd9V                  1
3           8725750      4rLbkk5                  1
4           8725750      4rPorjK                  1
5           8725750       5KQxVM                  1
6           8725750       606XOl                  1
7           8725750      6r5Ge3r                  1
8           8725750      6rB1NXx                  1
9           8725750      6rPGLR4                  1
10          8725750       6w8lOR                  1
11          8725750      8MP1qRz                  1
12          8725750       8r03az                  1
13          8725750       8rYAPO                  1
14          8725750       8vj3v3                  1
15          8725750      96XeV8P                  1
16          8725750      96Z5lP5                  1
17          8725750      96Zqw15

# Vistas

### Trabajo con la vistas, para reducirlas y generar avisos vistos, pero no postulados

In [18]:
vistas['timestamp'] = pd.to_datetime(vistas['timestamp'])
vistas['dateVista'] = pd.DatetimeIndex(vistas['timestamp']).date

In [20]:
vistasGrouped = vistas.groupby(['dateVista','idpostulante','idAviso']).count();
vistasGrouped = vistasGrouped.reset_index();

In [21]:
vistasGrouped.info

<bound method DataFrame.info of           dateVista idpostulante     idAviso  timestamp
0        2018-03-22      0z5JW1r  1112417455          1
1        2018-03-22      0zB02qr  1112317620          1
2        2018-03-22      0zB02qr  1112362627          2
3        2018-03-22      0zB02qr  1112384556          6
4        2018-03-22      0zB02qr  1112403061          2
5        2018-03-22      0zB02qr  1112410278          5
6        2018-03-22      0zB02qr  1112417329          2
7        2018-03-22      0zB1oA1  1112417467          1
8        2018-03-22      0zB2Kp9  1112408821          1
9        2018-03-22      0zB2bxr  1112246884          2
10       2018-03-22      0zB2bxr  1112352890          2
11       2018-03-22      0zB2bxr  1112355747          2
12       2018-03-22      0zB2bxr  1112407559          1
13       2018-03-22      0zB2bxr  1112409876          2
14       2018-03-22      0zB2eMa  1112362057          2
15       2018-03-22      0zB2eMa  1112419006          2
16       2018-03

In [39]:
#vistasGroupedFinal = vistasGrouped.drop('timeVista',axis=1)

In [22]:
vistasGrouped['cantidadVistas'] = vistasGrouped['timestamp'];
vistasGrouped.drop('timestamp',axis=1,inplace=True)
vistasForMerge = vistasGrouped[['idpostulante','idAviso']]

# Generamos archivos con avisos no vistos

In [472]:
import datetime as dt

def generoNOAvisos(cantParaArmar):
    sumaNoPostulados = pd.DataFrame();
    for i in range(int(cantParaArmar)):
        print("--------------------- Iteracion de sample " + str(i) +" de "+ str(cantParaArmar) +"---------");
        vistasSample = vistasForMerge.sample(frac=0.1);
        postulacionesSample = avisosPostuladosGP.sample(frac=0.1);

        avisosNoVistos = pd.merge(postulacionesSample,vistasSample,left_on='idaviso',right_on='idAviso',how='right', indicator=True)
        avisosNoVistosPosta = avisosNoVistos[avisosNoVistos['_merge'] == 'right_only']
        avisosNoVistosFinal = avisosNoVistosPosta.drop(['idpostulante_x','cantPostulaciones','_merge'],axis=1)
        avisosNoVistosFinal['idaviso'] = avisosNoVistosFinal['idAviso'];
        avisosNoVistosFinal['idpostulante'] = avisosNoVistosFinal['idpostulante_y'];
        avisosNoVistosFinal.drop(['idpostulante_y','idAviso'],axis=1,inplace=True);
        sumaNoPostulados = pd.concat([sumaNoPostulados,avisosNoVistosFinal])
    return sumaNoPostulados;


def generoArchivosNOPostulado(cantArchivos,cantInicial,cantIteracionesPorArhivo):
    print("--- HORA DE INICIO PROCESO TOTAL "+ str(dt.datetime.now()) + "-----");
    var hasta = int(cantArchivos) + int(cantInicial);
    for i in range(int(cantInicial),hasta):
        print("---------- Generando archivos " + str(i) +" de "+ str(cantArchivos) +"-------");
        print("---------- HORA DE INICIO "+ str(dt.datetime.now()) + "-------");
        sumaTotalNoAvisos = generoNOAvisos(int(cantIteracionesPorArhivo));
        sumaTotalNoAvisosSinDuplicados = sumaTotalNoAvisos.drop_duplicates().reset_index()
        sumaTotalNoAvisosSinDuplicados['sepostulo'] = 0
        sumaTotalNoAvisosSinDuplicados = sumaTotalNoAvisosSinDuplicados[['idpostulante','idaviso','sepostulo']]
        sumaTotalNoAvisosSinDuplicados.to_csv('nopostulados/nopostulados-'+str(i)+'.csv');
        print("---------- HORA DE FIN "+ str(dt.datetime.now()) + "--------------------------");
    print("--- HORA DE FIN PROCESO TOTAL "+ str(dt.datetime.now()) + "-----");
        

In [1043]:
##Llamamos proceso

generoArchivosNOPostulado(20,15,20);

--- HORA DE INICIO PROCESO TOTAL 2018-06-22 01:34:19.238586-----
---------- Generando archivos 15 de 20-------
---------- HORA DE INICIO 2018-06-22 01:34:19.238586-------
--------------------- Iteracion de sample 0 de 20---------
--------------------- Iteracion de sample 1 de 20---------
--------------------- Iteracion de sample 2 de 20---------
--------------------- Iteracion de sample 3 de 20---------
--------------------- Iteracion de sample 4 de 20---------
--------------------- Iteracion de sample 5 de 20---------
--------------------- Iteracion de sample 6 de 20---------
--------------------- Iteracion de sample 7 de 20---------
--------------------- Iteracion de sample 8 de 20---------
--------------------- Iteracion de sample 9 de 20---------
--------------------- Iteracion de sample 10 de 20---------
--------------------- Iteracion de sample 11 de 20---------
--------------------- Iteracion de sample 12 de 20---------
--------------------- Iteracion de sample 13 de 20---------

In [24]:
avisosNoPostulado = pd.DataFrame();
for i in range(0,15):
    noPostulado = pd.read_csv('nopostulados/nopostulados-'+str(i)+'.csv')
    avisosNoPostulado = pd.concat([noPostulado,avisosNoPostulado]);

avisosNoPostulado.drop('Unnamed: 0',axis=1,inplace=True)
avisosNoPostulado = avisosNoPostulado.drop_duplicates()

In [26]:
nopostulados = avisosNoPostulado[['idpostulante','idaviso','sepostulo']];

In [27]:
avisosPostuladosGP['sepostulo'] = 1

In [28]:
avisosSample = avisosPostuladosGP.sample(frac=0.1)

In [29]:
datosMergeados = pd.concat([nopostulados,avisosSample])

In [30]:
datosMergeados = datosMergeados.fillna(0)

# Unimos con avisos con sus detalles

In [31]:
datosMergeadosConDetalle = pd.merge(datosMergeados,avisosDetalle,how='inner',on='idaviso');
datosMergeadosConDetalleFiltered = datosMergeadosConDetalle.drop(['titulo','descripcion','ciudad','mapacalle','nombre_area'],axis=1)

In [32]:
datosOHC = pd.concat([datosMergeadosConDetalleFiltered,pd.get_dummies(datosMergeadosConDetalleFiltered['nombre_zona'], prefix='zona')],axis=1)
datosOHC = pd.concat([datosOHC,pd.get_dummies(datosOHC['tipo_de_trabajo'], prefix='tipo')],axis=1)
datosOHC = pd.concat([datosOHC,pd.get_dummies(datosOHC['nivel_laboral'], prefix='nivel')],axis=1)

datosOHC.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona'],axis=1,inplace=True)

In [33]:
datosOHC['denominacion_empresa'].fillna('S/E', inplace=True)
datosOHC = pd.merge(datosOHC,cantAvisosPorEmpresa,how='inner',on='denominacion_empresa')
datosOHC.drop('denominacion_empresa',axis=1,inplace=True);

In [34]:
datosForTraining = datosOHC.drop(['sepostulo','idaviso','idpostulante'],axis=1)

# Probamos algoritmo

In [38]:
import pandas
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn import gaussian_process
from sklearn import neural_network
from sklearn.linear_model import LinearRegression, LogisticRegression



#Pre-procesamiento
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

In [39]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTraining, datosOHC['sepostulo'], test_size=0.7, random_state=7)

In [40]:
models = []
models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
models.append(('LR', linear_model.LinearRegression()))
models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
models.append(('BAYESIAN',linear_model.BayesianRidge()))
models.append(('SVR',svm.LinearSVR()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('MLP',tree.DecisionTreeRegressor()))
models.append(('NB', GaussianNB()))
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
 
TestModels = pd.DataFrame()
tmp = {}

results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, scoring='r2')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

RF10: 0.999927 (0.000036)
RF20: 0.999960 (0.000013)
RF50: 0.999966 (0.000009)
RF60: 0.999972 (0.000008)
KNN 2: 0.980356 (0.002552)
KNN 3: 0.978372 (0.001242)
LR: 0.999062 (0.000606)
RIDGE.05: 0.999062 (0.000606)
LASO.01: 0.999062 (0.000606)
LASOLARS.05: -0.000027 (0.000019)
BAYESIAN: 0.999062 (0.000606)
SVR: 0.999052 (0.000616)
DES-TREE: 1.000000 (0.000000)
MLP: 1.000000 (0.000000)
NB: -0.175106 (0.000010)
LR: -0.175106 (0.000010)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LDA: 1.000000 (0.000000)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [195]:
models = []
models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
models.append(('LR', linear_model.LinearRegression()))
models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
models.append(('BAYESIAN',linear_model.BayesianRidge()))
models.append(('SVR',svm.LinearSVR()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('MLP',tree.DecisionTreeRegressor()))
models.append(('NB', GaussianNB()))
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
 
TestModels = pd.DataFrame()
tmp = {}

results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, scoring='r2')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

RF10: 0.999965 (0.000003)
RF20: 0.999986 (0.000006)
RF50: 0.999991 (0.000004)
RF60: 0.999993 (0.000002)
KNN 2: 0.992723 (0.000696)
KNN 3: 0.991316 (0.000391)
LR: 0.999623 (0.000216)
RIDGE.05: 0.999623 (0.000215)
LASO.01: 0.999623 (0.000216)
LASOLARS.05: -0.000000 (0.000000)
BAYESIAN: 0.999623 (0.000216)
SVR: 0.999617 (0.000218)
DES-TREE: 1.000000 (0.000000)
MLP: 1.000000 (0.000000)
NB: 0.747843 (0.001135)
LR: -0.740777 (0.020453)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LDA: 1.000000 (0.000000)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


# Training Set

In [41]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

In [200]:
testingSet.columns

Index(['id', 'idaviso', 'idpostulante'], dtype='object')

In [42]:
testingSet['testSet'] =1;
datosMergeados['testSet'] = 0;

In [43]:
datosMergeadosConDetalle = pd.merge(datosMergeados,avisosDetalle,how='inner',on='idaviso');
testingSetConDetalle = pd.merge(testingSet,avisosDetallesCompletos,how='left',on='idaviso');
#
#
datosConTestSet = pd.concat([datosMergeadosConDetalle,testingSetConDetalle]);
datosConTestSetFiltrado = datosConTestSet.drop(['titulo','descripcion','ciudad','mapacalle','nombre_area'],axis=1)

In [44]:
datosConTestOHC = pd.concat([datosConTestSetFiltrado,pd.get_dummies(datosConTestSetFiltrado['nombre_zona'], prefix='zona')],axis=1)
datosConTestOHC = pd.concat([datosConTestOHC,pd.get_dummies(datosConTestOHC['tipo_de_trabajo'], prefix='tipo')],axis=1)
datosConTestOHC = pd.concat([datosConTestOHC,pd.get_dummies(datosConTestOHC['nivel_laboral'], prefix='nivel')],axis=1)

datosConTestOHC.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona'],axis=1,inplace=True)

In [45]:
datosConTestOHC['denominacion_empresa'].fillna('S/E', inplace=True)
datosConTestOHC = pd.merge(datosConTestOHC,cantAvisosPorEmpresa,how='left',on='denominacion_empresa')
datosConTestOHC.drop('denominacion_empresa',axis=1,inplace=True);

In [46]:
datosForTraining = datosConTestOHC[datosConTestOHC['testSet'] == 0]
datosForTest = datosConTestOHC[datosConTestOHC['testSet'] == 1]

In [47]:
datosForTrainingSinPostu = datosForTraining.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [48]:
datosForTestFinal = datosForTest.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [350]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosForTrainingSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(datosForTestFinal)

In [353]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, datosForTraining['sepostulo'], test_size=0.7, random_state=7)

In [357]:
models = []
models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
models.append(('LR', linear_model.LinearRegression()))
models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
#models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
models.append(('BAYESIAN',linear_model.BayesianRidge()))
models.append(('SVR',svm.LinearSVR()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('MLP',tree.DecisionTreeRegressor()))
models.append(('NB', GaussianNB()))
#models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))



for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba5-'+ name +'.csv', index=False)

C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


# Agregamos genero y Estudios

In [50]:
#Cargamos datasets

postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')

postulantesEducacionZip = pd.read_csv('resources/zip/fiuba_1_postulantes_educacion.csv')
postulantesGeneroZip = pd.read_csv('resources/zip/fiuba_2_postulantes_genero_y_edad.csv')

In [51]:
educacionTodas = pd.concat([postulantesEducacion,postulantesEducacionZip])
generoTodas = pd.concat([postulantesGenero,postulantesGeneroZip])

In [52]:
import datetime as dt

generoTodas['fechanacimiento'] = pd.to_datetime(generoTodas['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
generoTodas = generoTodas.dropna(subset=['fechanacimiento'])

hoy = pd.Timestamp(dt.datetime.now())
generoTodas['edad'] = (hoy - generoTodas['fechanacimiento']).astype('<m8[Y]')


In [53]:
generoTodasFiltered = generoTodas.loc[((generoTodas['edad'] >= 17) & (generoTodas['edad'] < 80 )), :]

In [54]:
postulantesGeneroOHC = pd.concat([generoTodasFiltered,pd.get_dummies(generoTodasFiltered['sexo'], prefix='sexo')],axis=1)
postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','sexo_FEM','sexo_MASC','sexo_NO_DECLARA','edad']]

postulantesGeneroOHC.drop_duplicates(subset='idpostulante',inplace=True)

# CAMBIAR FORMA DE TRABAJAR LA EDUCACION

In [401]:
postulantesEducacionOHC = pd.concat([educacionTodas,pd.get_dummies(educacionTodas['estado'], prefix='estado')],axis=1)
postulantesEducacionOHC = pd.concat([postulantesEducacionOHC,pd.get_dummies(postulantesEducacionOHC['nombre'], prefix='nombre')],axis=1)

postulantesEducacionOHC.drop(['nombre'],axis=1, inplace=True)
postulantesEducacionOHC.drop(['estado'],axis=1, inplace=True)

In [440]:
postulantesEducacionOHCGP = postulantesEducacionOHC.groupby('idpostulante').sum()
postulantesEducacionOHCGP = postulantesEducacionOHCGP.reset_index()

In [454]:
##CARGO LOS ELEMENTOS DEL TEST ANTERIOR
#datosForTraining
#datosForTest

datosForTraining = datosConTestOHC[datosConTestOHC['testSet'] == 0]
datosForTest = datosConTestOHC[datosConTestOHC['testSet'] == 1]

In [455]:
datosForTrainingConEducacion = pd.merge(datosForTraining,postulantesEducacionOHCGP,on='idpostulante',how='inner')
datosForTrainingConTodo = pd.merge(datosForTrainingConEducacion,postulantesGeneroOHC,on='idpostulante',how='inner')

In [456]:
datosForTestConEducacion = pd.merge(datosForTest,postulantesEducacionOHCGP,on='idpostulante',how='left')
datosForTestConTodo = pd.merge(datosForTestConEducacion,postulantesGeneroOHC,on='idpostulante',how='left')

In [457]:
datosForTestConTodo.info

<bound method DataFrame.info of        cantPostulaciones       id     idaviso  idpais idpostulante  sepostulo  \
0                    NaN      0.0      739260     1.0       6M9ZQR        NaN   
1                    NaN      1.0      739260     1.0       6v1xdL        NaN   
2                    NaN      2.0      739260     1.0       ezRKm9        NaN   
3                    NaN      3.0      758580     1.0       1Q35ej        NaN   
4                    NaN      4.0      758580     1.0       EAN4J6        NaN   
5                    NaN      5.0      758580     1.0       8R6pzR        NaN   
6                    NaN      6.0      776420     1.0       aZJ2XN        NaN   
7                    NaN      7.0      776420     1.0       Nmpo3J        NaN   
8                    NaN      8.0      776420     1.0       eVqWar        NaN   
9                    NaN      9.0      820850     1.0       6ZBD33        NaN   
10                   NaN     10.0      820850     1.0       1j4RNj        NaN

In [458]:
datosForTrainingSinPostu = datosForTrainingConTodo.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [459]:
datosForTestConTodoFiltrado = datosForTestConTodo.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [460]:
datosForTestConTodoFiltrado.columns.size

46

In [461]:
datosForTrainingSinPostu.columns.size

46

In [464]:
datosForTrainingSinPostu.drop('fechanacimiento',axis=1,inplace=True)

In [463]:
datosForTestConTodoFiltrado.drop('fechanacimiento',axis=1,inplace=True)

In [465]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosForTrainingSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(datosForTestConTodoFiltrado)

In [466]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, datosForTrainingConTodo['sepostulo'], test_size=0.7, random_state=7)

In [469]:
models = []
#models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
#models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
#models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
#models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
#models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
#models.append(('LR', linear_model.LinearRegression()))
#models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
#models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
#models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
#models.append(('BAYESIAN',linear_model.BayesianRidge()))
#models.append(('SVR',svm.LinearSVR()))
#models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
#models.append(('NB', GaussianNB()))
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))



for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba6-'+ name +'.csv', index=False)

#  Agregamos la cantidad de vistas de un aviso

In [55]:
vistasZIP = pd.read_csv('resources/zip/fiuba_3_vistas.csv')

In [56]:
vistasZIP['timestamp'] = pd.to_datetime(vistasZIP['timestamp'])
vistasZIP['dateVista'] = pd.DatetimeIndex(vistasZIP['timestamp']).date

In [57]:
vistasZIP['cantidadVistos'] = 1;
vistasZIP.drop('timestamp',axis=1,inplace=True)

In [58]:
vistasZIPGP = vistasZIP.groupby(['idAviso','idpostulante']).sum()

In [59]:
vistasZIPGP.head(20)

cantidadVistos
idAviso idpostulante                
18      BolNL                      3
48375   RwVdKR                     1
169730  1KjXB                      2
        2AKzxa                     2
        6LJ64                      1
        6kr39v                     1
        9VzdP                      3
        9zGM9                      1
        BVvYz                      1
        GzQZaK                     1
        JOYa2J                     2
        LLBE5                      2
        Lm2Zkb                     3
        MNGJOx                     1
        NwbWx                      2
        QdQ5A4                     2
        Qddmqe                     1
        RwBL9Y                     2
        VBprEN                     1
        Y0qD8J                     1

In [60]:
vistasZIPGP = vistasZIPGP.reset_index()

In [61]:
vistas['cantidadVistos'] =1;

In [62]:
vistasGP = vistas.groupby(['idAviso','idpostulante']).sum()

In [63]:
vistasGP = vistasGP.reset_index()

In [64]:
datosForTraining = datosConTestOHC[datosConTestOHC['testSet'] == 0]
datosForTest = datosConTestOHC[datosConTestOHC['testSet'] == 1]

In [65]:
vistasGP['idaviso']  = vistasGP['idAviso']
vistasGP.drop('idAviso',axis=1,inplace=True);

In [66]:
vistasZIPGP['idaviso']  = vistasZIPGP['idAviso']
vistasZIPGP.drop('idAviso',axis=1,inplace=True);

In [501]:
datosForTraining.columns

Index(['cantPostulaciones', 'id', 'idaviso', 'idpais', 'idpostulante',
       'sepostulo', 'testSet', 'zona_Buenos Aires (fuera de GBA)',
       'zona_Capital Federal', 'zona_Catamarca', 'zona_Ciudad de Mendoza',
       'zona_Cordoba', 'zona_Corrientes', 'zona_GBA Oeste',
       'zona_Gran Buenos Aires', 'zona_La Plata', 'zona_Mendoza',
       'zona_Neuquen', 'zona_Rosario', 'zona_San Juan', 'zona_Santa Cruz',
       'zona_Santa Fe', 'zona_Tucuman', 'tipo_Fines de Semana',
       'tipo_Full-time', 'tipo_Part-time', 'tipo_Pasantia',
       'tipo_Por Contrato', 'tipo_Por Horas', 'tipo_Primer empleo',
       'tipo_Teletrabajo', 'tipo_Temporario',
       'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'cantidadEmpresa'],
      dtype='object')

In [67]:
datosConVistas = pd.merge(datosForTraining, vistasGP,on=['idaviso','idpostulante'],how='inner')

In [68]:
testingConVistas = pd.merge(datosForTest,vistasZIPGP,on=['idaviso','idpostulante'],how='left')

In [69]:
testingConVistas['cantidadVistos'] = testingConVistas['cantidadVistos'].fillna(0)

In [70]:
testingConVistas['cantidadVistos'].unique()

array([  0.,   1.,   2.,   5.,   3.,   4.,   7.,   6.,   8.,  10.,  12.,
        21.,  15.,   9.,  14.,  13.,  38.,  16.,  30.,  11.,  19.,  20.,
        49.,  22.,  25.,  18.,  27.,  17.,  28.,  24.,  65.,  23.,  41.,
        34.,  33.,  37.,  51.])

In [518]:
testingConVistas.columns

Index(['cantPostulaciones', 'id', 'idaviso', 'idpais', 'idpostulante',
       'sepostulo', 'testSet', 'zona_Buenos Aires (fuera de GBA)',
       'zona_Capital Federal', 'zona_Catamarca', 'zona_Ciudad de Mendoza',
       'zona_Cordoba', 'zona_Corrientes', 'zona_GBA Oeste',
       'zona_Gran Buenos Aires', 'zona_La Plata', 'zona_Mendoza',
       'zona_Neuquen', 'zona_Rosario', 'zona_San Juan', 'zona_Santa Cruz',
       'zona_Santa Fe', 'zona_Tucuman', 'tipo_Fines de Semana',
       'tipo_Full-time', 'tipo_Part-time', 'tipo_Pasantia',
       'tipo_Por Contrato', 'tipo_Por Horas', 'tipo_Primer empleo',
       'tipo_Teletrabajo', 'tipo_Temporario',
       'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'cantidadEmpresa', 'cantidadVistos'],
      dtype='object')

In [73]:
datosConVistasSinPostu = datosConVistas.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)
testingConVistasFiltrado = testingConVistas.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet'],axis=1)

In [520]:
datosConVistasSinPostu.columns.size

32

In [521]:
testingConVistasFiltrado.columns.size

32

In [523]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosConVistasSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(testingConVistasFiltrado)

In [525]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, datosConVistas['sepostulo'], test_size=0.7, random_state=7)

In [526]:
models = []
#models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
#models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
#models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
#models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
#models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
#models.append(('LR', linear_model.LinearRegression()))
#models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
#models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
#models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
#models.append(('BAYESIAN',linear_model.BayesianRidge()))
#models.append(('SVR',svm.LinearSVR()))
#models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
#models.append(('NB', GaussianNB()))
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))



for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba7-'+ name +'.csv', index=False)

# Agregamos edades

In [ ]:
datosConVistas;
testingConVistas;

In [77]:
datosGeneroMerged = pd.merge(datosConVistas,postulantesGenero,how='left',on='idpostulante')

In [78]:
import datetime as dt

datosGeneroMerged['fechanacimiento'] = pd.to_datetime(datosGeneroMerged['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
datosGeneroMerged = datosGeneroMerged.dropna(subset=['fechanacimiento'])

hoy = pd.Timestamp(dt.datetime.now())
datosGeneroMerged['edad'] = (hoy - datosGeneroMerged['fechanacimiento']).astype('<m8[Y]')


In [79]:
datosGeneroMergedFiltered = datosGeneroMerged.loc[((datosGeneroMerged['edad'] >= 17) & (datosGeneroMerged['edad'] < 80 )), :]

In [80]:
postulantesGeneroOHC = pd.concat([datosGeneroMergedFiltered,pd.get_dummies(datosGeneroMergedFiltered['sexo'], prefix='sexo')],axis=1)

In [81]:
testingConVistasMerged = pd.merge(testingConVistas,postulantesGenero,how='left',on='idpostulante')

In [82]:
testingConVistasMerged['fechanacimiento'] = pd.to_datetime(testingConVistasMerged['fechanacimiento'], format='%Y-%m-%d', errors='coerce')

In [83]:
testingConVistasMerged['edad'] = (hoy - testingConVistasMerged['fechanacimiento']).astype('<m8[Y]')

In [84]:
testingConVistasMergedGeneroOHC = pd.concat([testingConVistasMerged,pd.get_dummies(testingConVistasMerged['sexo'], prefix='sexo')],axis=1)

In [85]:
postulantesGeneroOHC.drop('fechanacimiento',inplace=True,axis=1)

In [86]:
testingConVistasMergedGeneroOHC.drop('fechanacimiento',inplace=True,axis=1)

In [538]:
testingConVistasMergedGeneroOHC.info

<bound method DataFrame.info of        cantPostulaciones       id     idaviso  idpais idpostulante  sepostulo  \
0                    NaN      0.0      739260     1.0       6M9ZQR        NaN   
1                    NaN      1.0      739260     1.0       6v1xdL        NaN   
2                    NaN      2.0      739260     1.0       ezRKm9        NaN   
3                    NaN      3.0      758580     1.0       1Q35ej        NaN   
4                    NaN      4.0      758580     1.0       EAN4J6        NaN   
5                    NaN      5.0      758580     1.0       8R6pzR        NaN   
6                    NaN      6.0      776420     1.0       aZJ2XN        NaN   
7                    NaN      7.0      776420     1.0       Nmpo3J        NaN   
8                    NaN      8.0      776420     1.0       eVqWar        NaN   
9                    NaN      9.0      820850     1.0       6ZBD33        NaN   
10                   NaN     10.0      820850     1.0       1j4RNj        NaN

In [88]:
datosConVistasSinPostu = postulantesGeneroOHC.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo'],axis=1)
testingConVistasFiltrado = testingConVistasMergedGeneroOHC.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo'],axis=1)

In [588]:
datosConVistasSinPostu.columns.size

36

In [587]:
testingConVistasFiltrado.columns.size

36

In [91]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosConVistasSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(testingConVistasFiltrado)

In [591]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, postulantesGeneroOHC['sepostulo'], test_size=0.7, random_state=7)

In [592]:
models = []
#models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
#models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
#models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
#models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
#models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
#models.append(('LR', linear_model.LinearRegression()))
#models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
#models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
#models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
#models.append(('BAYESIAN',linear_model.BayesianRidge()))
#models.append(('SVR',svm.LinearSVR()))
#models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
#models.append(('NB', GaussianNB()))
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))



for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba8-'+ name +'.csv', index=False)

# Mejoramos estudios

In [598]:
#educacionOHC = pd.concat([postulantesEducacion,pd.get_dummies(postulantesEducacion['nombre'], prefix='educacion')],axis=1)

In [602]:
postulantesEducacion.head()

,idpostulante,nombre,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [92]:
postulantesEducacion['nombreEstado'] = postulantesEducacion['nombre'] +" - " + postulantesEducacion['estado']

In [93]:
educacionOHC = pd.concat([postulantesEducacion,pd.get_dummies(postulantesEducacion['nombreEstado'], prefix='educacion')],axis=1)

In [94]:
educacionOHC.drop(['nombre','estado','nombreEstado'],axis=1,inplace=True)

In [95]:
educacionOHCGP = educacionOHC.groupby('idpostulante').sum()

In [96]:
educacionOHCGP = educacionOHCGP.reset_index()

In [97]:
##Mergeamos
#postulantesGeneroOHC
#testingConVistasMergedGeneroOHC

datosTrainingConEstudios = pd.merge(postulantesGeneroOHC,educacionOHCGP,how='inner',on='idpostulante')

In [98]:
datosTestingConEstudios = pd.merge(testingConVistasMergedGeneroOHC,educacionOHCGP,how='left',on='idpostulante')

In [99]:
datosConVistasSinPostu = datosTrainingConEstudios.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo'],axis=1)
testingConVistasFiltrado = datosTestingConEstudios.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo'],axis=1)

In [629]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosConVistasSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(testingConVistasFiltrado)

In [630]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, datosTrainingConEstudios['sepostulo'], test_size=0.7, random_state=7)

In [631]:
models = []
#models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
#models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
#models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
#models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
#models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
#models.append(('LR', linear_model.LinearRegression()))
#models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
#models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
#models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
#models.append(('BAYESIAN',linear_model.BayesianRidge()))
#models.append(('SVR',svm.LinearSVR()))
#models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
#models.append(('NB', GaussianNB()))
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))



for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba9-'+ name +'.csv', index=False)

# Agregamos nombre area (Hasta ahora la mejor)

In [637]:
from sklearn.feature_extraction.text import TfidfVectorizer

TFIDF = TfidfVectorizer()
areaConTFIDF = TFIDF.fit_transform(avisosDetalle['nombre_area']);


In [650]:
from sklearn.base import BaseEstimator, TransformerMixin

class NicoTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        self

    def transform(self, X, y=None):
        return X

    def fit(self, X, y=None):
        return self

In [769]:
#Creamos un pipeline

from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion


def getTextCols(df):
    return df[['titulo','nombre_area','descripcion']];

def getResto(df):
    return df.drop(['titulo','nombre_area','descripcion'],axis=1);


class ArrayCaster(BaseEstimator, TransformerMixin):
  def fit(self, x, y=None):
    return self

  def transform(self, data):
    return  np.array(list(data))


vec = make_union(*[
    make_pipeline(FunctionTransformer(getTextCols, validate=False),TfidfVectorizer())
])

In [100]:
detalleTodas = pd.concat([avisosDetalle, avisosDetallesCompletos])

In [101]:
detalleTodas['cantidadNombre'] = 1
detalleTodas['cantidadNombre'].fillna('Otros')
detalleTodasGP = detalleTodas.groupby('nombre_area').sum()

In [102]:
detalleTodasGP = detalleTodasGP.reset_index()
detalleTodasFiltered = detalleTodasGP[['nombre_area','cantidadNombre']]

In [984]:
detalleTodasFiltered.head()

,nombre_area,cantidadNombre
0,Abastecimiento,152
1,Administracion de Seguros,46
2,Administración,2215
3,Administración de Base de Datos,29
4,Administración de Personal,376


In [104]:
avisosDetalleF = avisosDetalle[['idaviso','nombre_area']]
trainingMergeadoConDetalle = pd.merge(datosTrainingConEstudios,avisosDetalleF,on='idaviso',how='inner')

In [105]:
adf = avisosDetallesCompletos[['idaviso','nombre_area']]
testingMergeadoConDetalle = pd.merge(datosTestingConEstudios,adf,on='idaviso',how='left')

In [106]:
testingConCantNombre = pd.merge(testingMergeadoConDetalle,detalleTodasFiltered,on='nombre_area',how='left')
trainingConCantNombre = pd.merge(trainingMergeadoConDetalle,detalleTodasFiltered,on='nombre_area',how='inner')

In [1014]:
testingConCantNombre.columns.size

67

In [1015]:
trainingConCantNombre.columns.size

67

In [109]:
datosConVistasSinPostu = trainingConCantNombre.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area'],axis=1)
testingConVistasFiltrado = testingConCantNombre.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area'],axis=1)

In [110]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosConVistasSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(testingConVistasFiltrado)

In [111]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, trainingConCantNombre['sepostulo'], test_size=0.7, random_state=7)

In [116]:
datosConVistasSinPostu.columns.size

59

In [113]:
models = []
#models.append(('RF10', RandomForestRegressor(n_estimators=10, max_features='sqrt')))
#models.append(('RF20', RandomForestRegressor(n_estimators=20, max_features='sqrt')))
#models.append(('RF50',  RandomForestRegressor(n_estimators=50, max_features='sqrt')))
#models.append(('RF60',  RandomForestRegressor(n_estimators=60, max_features='sqrt')))
models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('RF100',  RandomForestClassifier(n_estimators=100, max_features='sqrt')))
#models.append(('KNN 2', KNeighborsClassifier(n_neighbors=2)))
#models.append(('KNN 3', KNeighborsClassifier(n_neighbors=3)))
#models.append(('LR', linear_model.LinearRegression()))
#models.append(('RIDGE.05', linear_model.Ridge (alpha = .5)))
#models.append(('LASO.01', linear_model.Ridge (alpha = .1)))
#models.append(('LASOLARS.05',linear_model.LassoLars (alpha = 0.5)))
#models.append(('BAYESIAN',linear_model.BayesianRidge()))
#models.append(('SVR',svm.LinearSVR()))
#models.append(('DES-TREE',tree.DecisionTreeRegressor()))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))
#models.append(('NB', GaussianNB()))
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))



for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    #archivo.to_csv('entregas/'+'prueba10-'+ name +'.csv', index=False)
    #archivo.to_csv('entregas/'+'prueba15-'+ name +'.csv', index=False)
    archivo.to_csv('entregas/'+'pruebaRE-ENTRENAMIENTO-15-2'+ name +'.csv', index=False)

# Pruebo eliminando duplicados en training set -- DA PEOR

In [1036]:
trainingConCantNombreSinDuplicados = trainingConCantNombre.drop_duplicates(subset=['idaviso','idpostulante'])

In [1037]:
datosConVistasSinPostu = trainingConCantNombreSinDuplicados.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area'],axis=1)
testingConVistasFiltrado = testingConCantNombre.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area'],axis=1)

In [1038]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosConVistasSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(testingConVistasFiltrado)

In [1039]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, trainingConCantNombreSinDuplicados['sepostulo'], test_size=0.7, random_state=7)

In [1040]:
models = []

models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))

for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba11-'+ name +'.csv', index=False)
    
    
    

# Agregamos titulo

In [1079]:
#Mala prueba

In [1046]:
#testingConCantNombre
#trainingConCantNombre


adf = avisosDetallesCompletos[['idaviso','titulo']]
testingConTitulo = pd.merge(testingConCantNombre,adf,on='idaviso',how='left')

In [1047]:
avisosDetalleF = avisosDetalle[['idaviso','titulo']]
trainingConTitulo = pd.merge(trainingConCantNombre,avisosDetalleF,on='idaviso',how='inner')

In [1053]:
avisosDetalleF['cantidadTitulo'] = 1
avg = avisosDetalleF.groupby('titulo').sum()

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1060]:
adf['cantidadTitulo'] = 1
adfg = adf.groupby('titulo').sum()

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1062]:
adfg = adfg.reset_index();
adfg.drop('idaviso',inplace=True,axis=1)

In [1056]:
avg.sort_values('cantidadTitulo',ascending=False)

,idaviso,cantidadTitulo
titulo,,
Analista Contable,35593204537,32
Administrativo Contable,28921237557,26
Analista de Testing manual Ssr. // Zona Recoleta CABA,22246263586,20
Vendedor,22246291121,20
Analistas de Testing Ssr/Sr con Automatización // MENDOZA,22246262600,20
Desarrolladores .NET C# nivel Jr. / de 6 a 18 meses de exp,21133757680,19
Analista de inteligencia comercial,21133239793,19
Ejecutivo de Ventas,20018610334,18
Desarrolladores JAVA SOA / JBOSS RED HAT // Zona Recoleta,20021759762,18


In [1058]:
avgFinal = avg.reset_index();
avgFinal.drop('idaviso',inplace=True,axis=1)

In [1059]:
trainingConCantTitulo =pd.merge(trainingConTitulo,avgFinal,on='titulo',how='inner')

In [1063]:
testingConCantTitulo = pd.merge(testingConTitulo,adfg,on='titulo',how='left')

In [1067]:
testingConCantTitulo.drop('titulo',axis=1,inplace=True)
trainingConCantTitulo.drop('titulo',axis=1,inplace=True)

In [1074]:
trainingConCantTitulo.columns

Index(['cantPostulaciones', 'id', 'idaviso', 'idpais', 'idpostulante',
       'sepostulo', 'testSet', 'zona_Buenos Aires (fuera de GBA)',
       'zona_Capital Federal', 'zona_Catamarca', 'zona_Ciudad de Mendoza',
       'zona_Cordoba', 'zona_Corrientes', 'zona_GBA Oeste',
       'zona_Gran Buenos Aires', 'zona_La Plata', 'zona_Mendoza',
       'zona_Neuquen', 'zona_Rosario', 'zona_San Juan', 'zona_Santa Cruz',
       'zona_Santa Fe', 'zona_Tucuman', 'tipo_Fines de Semana',
       'tipo_Full-time', 'tipo_Part-time', 'tipo_Pasantia',
       'tipo_Por Contrato', 'tipo_Por Horas', 'tipo_Primer empleo',
       'tipo_Teletrabajo', 'tipo_Temporario',
       'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'cantidadEmpresa', 'cantidadVistos',
       'sexo', 'edad', 'sexo_FEM', 'sexo_MASC', 'sexo_NO_DECLARA',
       'educacion_Doctorado - Abandonado', 'educacion_Doctorado - En Curso',

In [1068]:
datosConVistasSinPostu = trainingConCantTitulo.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area'],axis=1)
testingConVistasFiltrado = testingConCantTitulo.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area'],axis=1)

In [1073]:
testingConVistasFiltrado.drop('index',axis=1,inplace=True)

In [1075]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosConVistasSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(testingConVistasFiltrado)

In [1076]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, trainingConCantTitulo['sepostulo'], test_size=0.7, random_state=7)

In [1077]:
models = []

models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))

for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba12-'+ name +'.csv', index=False)

### Prueba 2

# Pasamos edades a Rango -- dio Peor

In [ ]:
trainingConCantNombre
testingConCantNombre

In [1081]:
trainingConCantNombre.head()

,cantPostulaciones,id,idaviso,idpais,idpostulante,sepostulo,testSet,zona_Buenos Aires (fuera de GBA),zona_Capital Federal,zona_Catamarca,...,educacion_Secundario - En Curso,educacion_Secundario - Graduado,educacion_Terciario/Técnico - Abandonado,educacion_Terciario/Técnico - En Curso,educacion_Terciario/Técnico - Graduado,educacion_Universitario - Abandonado,educacion_Universitario - En Curso,educacion_Universitario - Graduado,nombre_area,cantidadNombre
0,0.0,NaN,1112461512,1.0,YjoolD,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,Tecnologia / Sistemas,947
1,0.0,NaN,1112461512,1.0,YjoolD,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,Tecnologia / Sistemas,947
2,0.0,NaN,1112461512,1.0,YjoolD,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,Tecnologia / Sistemas,947
3,0.0,NaN,1112461512,1.0,YjoolD,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,Tecnologia / Sistemas,947
4,0.0,NaN,1112461512,1.0,YjoolD,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,Tecnologia / Sistemas,947


In [1083]:
postulantesEdad = postulantesGenero[['fechanacimiento','idpostulante']]

In [1087]:
trainingConCantNombre['menor24'] = np.where(trainingConCantNombre['edad'] < 25, 1, 0);
trainingConCantNombre['menor31'] = np.where(trainingConCantNombre['edad'] < 31, 1, 0);
trainingConCantNombre['menor36'] = np.where(trainingConCantNombre['edad'] < 36, 1, 0);
trainingConCantNombre['menor40'] = np.where(trainingConCantNombre['edad'] < 40, 1, 0);
trainingConCantNombre['menor50'] = np.where(trainingConCantNombre['edad'] < 51, 1, 0);
trainingConCantNombre['mayor50'] = np.where(trainingConCantNombre['edad'] > 51, 1, 0);
trainingConCantNombre['resto']   = np.where(trainingConCantNombre['edad'].isnull(), 1, 0);

In [1089]:
testingConCantNombre['menor24'] = np.where(testingConCantNombre['edad'] < 25, 1, 0);
testingConCantNombre['menor31'] = np.where(testingConCantNombre['edad'] < 31, 1, 0);
testingConCantNombre['menor36'] = np.where(testingConCantNombre['edad'] < 36, 1, 0);
testingConCantNombre['menor40'] = np.where(testingConCantNombre['edad'] < 40, 1, 0);
testingConCantNombre['menor50'] = np.where(testingConCantNombre['edad'] < 51, 1, 0);
testingConCantNombre['mayor50'] = np.where(testingConCantNombre['edad'] > 51, 1, 0);
testingConCantNombre['resto']   = np.where(testingConCantNombre['edad'].isnull(), 1, 0);

In [1090]:
datosConVistasSinPostu = trainingConCantTitulo.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area','edad'],axis=1)
testingConVistasFiltrado = testingConCantTitulo.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area','edad'],axis=1)
testingConVistasFiltrado.drop('index',axis=1,inplace=True)

In [1091]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosConVistasSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(testingConVistasFiltrado)

In [1092]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, trainingConCantTitulo['sepostulo'], test_size=0.7, random_state=7)

In [1094]:
models = []

models.append(('RF100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))

for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba13-'+ name +'.csv', index=False)

# Prueba 14 - TF - IDF

In [1110]:
testingConCantNombre.drop(['menor24','menor31','menor36','menor40','menor50','mayor50','resto'],axis=1,inplace=True)
trainingConCantNombre.drop(['menor24','menor31','menor36','menor40','menor50','mayor50','resto'],axis=1,inplace=True)

ValueError: labels ['menor24' 'menor31' 'menor36' 'menor40' 'menor50' 'mayor50' 'resto'] not contained in axis

In [1098]:
def limpiamosTexto(text):
    text

In [1114]:
avisosDetalle['descripcionSinHTML'] = avisosDetalle['descripcion'].str.strip()

In [1164]:
#elimino tags de descripcion
avisosDetalle['descripcionSinHTML'] = avisosDetalle['descripcion'].str.replace(r"<\/?\w+.*?>","");
avisosDetalle['descripcionSinHTML'] = avisosDetalle['descripcionSinHTML'].str.replace(r"\s{2,}"," ");

In [1160]:
#import nltk
#nltk.download()
#from nltk.corpus import stopwords
#stop = stopwords.words('spanish')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [1165]:
avisosDetalle['descripcionSinHTML'] =avisosDetalle['descripcionSinHTML'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [1167]:
avisosDetalle['descripcionSinHTML'].head(5)

0    Buscamos Responsable Producto incorporarse dep...
1    Para cliente, concesionario Chevrolet ubicado ...
2    Importante Estudio Auditoria Consultoria ubica...
3    Chevron is multinational company in the energy...
4    Importante empresa energías renovables, encuen...
Name: descripcionSinHTML, dtype: object

In [1171]:
tvec = TfidfVectorizer(min_df=.0025, max_df=.1,stop_words='english')
tvec_weights = tvec.fit_transform(avisosDetalle['descripcionSinHTML'])
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
top100 = weights_df.sort_values(by='weight', ascending=False).head(100)

In [1175]:
#datosMergeadosConDetalle.head()
datosMergeadosPrueba = datosMergeadosConDetalle[['sepostulo','descripcion','idaviso','titulo']]

In [1179]:
datosMergeadosPrueba['descripcion'] =datosMergeadosPrueba['descripcion'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1185]:
estimators = [("tf_idf", TfidfVectorizer()), 
              ("ridge", linear_model.Ridge())]
model = Pipeline(estimators)

In [1187]:
model.fit(datosMergeadosPrueba['descripcion'],datosMergeadosPrueba['sepostulo'])

Pipeline(memory=None,
     steps=[('tf_idf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [1189]:
testingSetConDetallePrueba =testingSetConDetalle[['descripcion','idaviso','titulo']]

In [1201]:
testingSetConDetallePrueba['descripcion'] = testingSetConDetallePrueba['descripcion'].fillna(" ")
#elimino tags de descripcion
testingSetConDetallePrueba['descripcion'] = testingSetConDetallePrueba['descripcion'].str.replace(r"<\/?\w+.*?>","");
testingSetConDetallePrueba['descripcion'] = testingSetConDetallePrueba['descripcion'].str.replace(r"\s{2,}"," ");

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [1195]:
testingSetConDetallePrueba['descripcion']

0        Nuestro cliente es una empresa nac. ubicada en...
1        Nuestro cliente es una empresa nac. ubicada en...
2        Nuestro cliente es una empresa nac. ubicada en...
3         TeleTech, the worlwide leader in outsourced c...
4         TeleTech, the worlwide leader in outsourced c...
5         TeleTech, the worlwide leader in outsourced c...
6        Somos una Consultora en Management, Transforma...
7        Somos una Consultora en Management, Transforma...
8        Somos una Consultora en Management, Transforma...
9        Importante compañía de packaging interesada en...
10       Importante compañía de packaging interesada en...
11       Importante compañía de packaging interesada en...
12       Importante compañía de packaging interesada en...
13       Requerimos Profesionales de las carreras de si...
14       Requerimos Profesionales de las carreras de si...
15       Requerimos Profesionales de las carreras de si...
16       Importante empresa de sistemas nos encomendó l.

In [1202]:
testingSetConDetallePrueba['descripcion'] =testingSetConDetallePrueba['descripcion'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1203]:
result= model.predict(testingSetConDetallePrueba['descripcion'])

In [1205]:
archivo = pd.DataFrame()
archivo['id'] = testingSet.loc[:,['id']]
archivo['resultDescripcion'] = pruebas

In [1206]:
testingConCantNombre['resultDescripcion'] = pruebas

In [1209]:
trainingConCantNombreDetalle = pd.merge(trainingConCantNombre,avisosDetalle[['idaviso','descripcion']],left_on='idaviso',right_on='idaviso',how='inner')

In [1211]:
trainingConCantNombreDetalle['descripcion'] = trainingConCantNombreDetalle['descripcion'].fillna(" ")
#elimino tags de descripcion
trainingConCantNombreDetalle['descripcion'] = trainingConCantNombreDetalle['descripcion'].str.replace(r"<\/?\w+.*?>","");
trainingConCantNombreDetalle['descripcion'] = trainingConCantNombreDetalle['descripcion'].str.replace(r"\s{2,}"," ");

trainingConCantNombreDetalle['descripcion'] =trainingConCantNombreDetalle['descripcion'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

resultT= model.predict(trainingConCantNombreDetalle['descripcion'])

In [1212]:
trainingConCantNombreDetalle['resultDescripcion'] = resultT

In [1217]:
datosConVistasSinPostu = trainingConCantNombreDetalle.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area','edad'],axis=1)
testingConVistasFiltrado = testingConCantNombre.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','sexo','nombre_area','edad'],axis=1)
#testingConVistasFiltrado.drop('index',axis=1,inplace=True)

ValueError: labels ['descripcion'] not contained in axis

In [1224]:
datosConVistasSinPostu.drop('descripcion',axis=1,inplace=True)

In [1225]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosConVistasSinPostu)
datosForTestFinalSinNulos = imp.fit_transform(testingConVistasFiltrado)

In [1226]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, trainingConCantNombreDetalle['sepostulo'], test_size=0.7, random_state=7)

In [1229]:
models = []

models.append(('RFC100',  RandomForestClassifier(n_estimators=100, max_features='sqrt')))
models.append(('RFR100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
models.append(('DES-TREE',tree.DecisionTreeRegressor()))

for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = np.around(pruebas, decimals=0)
    archivo.to_csv('entregas/'+'prueba16-'+ name +'.csv', index=False)